In [1]:
# coding: utf-8

# In[1]:

# !pip install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl numpy matplotlib torchtext

# In[2]:

# Standard PyTorch imports
import numpy as np

from tensorflow.contrib.layers import layer_norm
import nn_utils

# For plots
#get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt

import tensorflow as tf
real = 1
BATCH_SIZE = 16
#!conda install torchtext spacy
# !python -m spacy download en
# !python -m spacy download de
if real:
  from torchtext import data
  from torchtext import datasets
  import tqdm
  import re
  import spacy

  spacy_de = spacy.load('de')
  spacy_en = spacy.load('en')

  url = re.compile('(<url>.*</url>)')

  def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(url.sub('@URL@', text))]

  def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(url.sub('@URL@', text))]

  # Testing IWSLT
  DE = data.Field(
      tokenize=tokenize_de,
      init_token='<bos>',
      eos_token='<eos>',
      include_lengths=True)
  EN = data.Field(
      tokenize=tokenize_en,
      init_token='<bos>',
      eos_token='<eos>',
      include_lengths=True)

  train, val, test = datasets.IWSLT.splits(
      exts=('.de', '.en'), fields=(DE, EN))

  train_it = data.Iterator(
      train,
      batch_size=BATCH_SIZE,
      sort_within_batch=True,
      train=True,
      repeat=False,
      shuffle=True)
  MIN_WORD_FREQ = 10
  MAX_NUM_WORDS = 3000
  DE.build_vocab(train.src, min_freq=MIN_WORD_FREQ, max_size=MAX_NUM_WORDS)
  EN.build_vocab(train.trg, min_freq=MIN_WORD_FREQ, max_size=MAX_NUM_WORDS)

  num_wds_input = len(DE.vocab.itos)
  num_wds_output = len(EN.vocab.itos)
else:
  num_wds_input = 1004


class masked_softmax:
  def __init__(self, v, mask, dim=2):
    #bs, query dimension, key dimension
    v_mask = v * mask
    v_max = tf.reduce_max(v_mask, dim, keep_dims=True)
    v_stable = v_mask - v_max

    v_exp = tf.exp(v_stable) * mask
    v_exp_sum = tf.reduce_sum(v_exp, dim, keep_dims=True)
    self.v_mask, self.v_max, self.v_stable, self.v_exp, self.v_exp_sum = \
        v_mask, v_max, v_stable, v_exp, v_exp_sum
    self.output = v_exp / (v_exp_sum + 1e-20)


class Encoder:
  def __init__(self, num_wds, wd_ind, mask, ndims=64, n_layers=6):
    self.num_wds = num_wds
    self.wd_ind = wd_ind
    self.mask = mask
    self.length = tf.shape(self.wd_ind)[1]
    self.wd_emb = tf.Variable(
        tf.random_uniform([self.num_wds, ndims], minval=-1, maxval=1.))
    self.wd_vec = tf.nn.embedding_lookup(self.wd_emb, wd_ind)
    self.pos = tf.reshape(
        tf.range(tf.cast(self.length, tf.float32), dtype=tf.float32),
        (1, -1, 1))
    self.divider_exponent = tf.reshape(
        tf.range(tf.cast(ndims // 2, tf.float32)),
        (1, 1, -1)) * 2. / tf.cast(ndims, tf.float32)
    self.divider = tf.pow(10000., self.divider_exponent)
    self.input_to_sinusoids = self.pos / self.divider
    self.pos_sin = tf.sin(self.input_to_sinusoids)
    self.pos_cos = tf.cos(self.input_to_sinusoids)
    # self.position = tf.reshape(
    #     tf.range(tf.cast(self.length, tf.float32), dtype=tf.float32) / 10000,
    #     (1, -1, 1))
    self.position = tf.concat((self.pos_sin, self.pos_cos), -1)
    self.w_tilde = embedding = self.wd_vec + self.position
    self.encoding = []
    self.attentionLayers = []
    for _ in range(n_layers):
      attentionLayer = AttentionLayer(embedding, mask)
      embedding = attentionLayer.output
      self.encoding.append(embedding)
      self.attentionLayers.append(attentionLayer)


class AttentionLayer:
  def __init__(self, X, mask, X_decode=None, decode_mask=None, ff_layer=True):
    bs, length, ndim = [v.value for v in X.shape]
    self.X = X
    if X_decode is None:
      self.q, self.k, self.v = [
          tf.tanh(tf.layers.dense(X, ndim)) for _ in range(3)
      ]
      decode_mask = mask
    else:
      self.q = tf.tanh(tf.layers.dense(X_decode, ndim))
      self.k, self.v = [tf.tanh(tf.layers.dense(X, ndim)) for _ in range(2)]
    #batch, attention queries, attention keys, embeddings
    self.q_expanded = tf.expand_dims(self.q, 2)
    self.k_expanded = tf.expand_dims(self.k, 1)
    self.v_expanded = tf.expand_dims(self.v, 1)
    self.s_raw = tf.reduce_sum(self.q_expanded * self.k_expanded, -1)
    self.mask = tf.expand_dims(decode_mask, 2) * tf.expand_dims(mask, 1)
    self.masked_softmax = masked_softmax(self.s_raw, self.mask)
    self.s = self.masked_softmax.output
    self.a = tf.expand_dims(self.s * self.mask, -1) * self.v_expanded
    #A is shape bs, query, key, emb
    self.a_compressed = tf.reduce_sum(self.a, 2)
    if X_decode is None:
      self.e = layer_norm(self.a_compressed + X)
    else:
      self.e = layer_norm(self.a_compressed + X_decode)
    if ff_layer:
      self.output = layer_norm(tf.layers.dense(self.e, ndim) + self.e)
    else:
      self.output = self.e


class Decoder:
  def __init__(self, num_wds, wd_ind, mask, encoder, ndims=20, n_layers=6):
    self.num_wds = num_wds
    self.wd_ind = wd_ind
    self.mask = mask
    self.encoder = encoder
    self.length = tf.shape(self.wd_ind)[1]
    self.wd_emb = tf.Variable(
        tf.random_uniform([self.num_wds, ndims], minval=-1, maxval=1.))
    self.wd_vec = tf.nn.embedding_lookup(self.wd_emb, wd_ind)
    self.pos = tf.reshape(
        tf.range(tf.cast(self.length, tf.float32), dtype=tf.float32),
        (1, -1, 1))
    self.divider_exponent = tf.reshape(
        tf.range(tf.cast(ndims // 2, tf.float32)),
        (1, 1, -1)) * 2. / tf.cast(ndims, tf.float32)
    self.divider = tf.pow(10000., self.divider_exponent)
    self.input_to_sinusoids = self.pos / self.divider
    self.pos_sin = tf.sin(self.input_to_sinusoids)
    self.pos_cos = tf.cos(self.input_to_sinusoids)
    # self.position = tf.reshape(
    #     tf.range(tf.cast(self.length, tf.float32), dtype=tf.float32) / 10000,
    #     (1, -1, 1))
    self.position = tf.concat((self.pos_sin, self.pos_cos), -1)
    self.w_tilde = embedding = self.wd_vec + self.position
    self.decoding = []
    self.self_attentions = []
    self.encoder_attentions = []
    for l_idx in range(n_layers):
      attn = AttentionLayer(embedding, mask, ff_layer=False)
      self.self_attentions.append(attn)
      encode_attn = AttentionLayer(encoder.encoding[l_idx], encoder.mask,
                                   attn.output, mask)
      self.encoder_attentions.append(encode_attn)
      embedding = encode_attn.output

    self.output_raw = tf.layers.dense(embedding, num_wds)
    #bs, word in sentence of target, embedding

    self.outsoftmax = masked_softmax(self.output_raw, tf.expand_dims(mask, -1), dim=2)
    self.output = self.outsoftmax.output

class Transformer:
  def __init__(self, num_wds):
    self.num_wds = num_wds
    n_layers = 6
    ndims = 256
    self.learning_rate = tf.placeholder(tf.float32, None)
    self.wd_ind_src = wd_ind_src = tf.placeholder(tf.int32, (None, None))
    self.wd_ind_trg = wd_ind_trg = tf.placeholder(tf.int32, (None, None))
    self.input_lengths = tf.placeholder(tf.int32, [None])
    self.output_lengths = tf.placeholder(tf.int32, [None])
    self.input_mask = tf.sequence_mask(
        self.input_lengths,
        maxlen=tf.shape(self.wd_ind_src)[-1],
        dtype=tf.float32)
    self.output_mask = tf.sequence_mask(
        self.output_lengths,
        maxlen=tf.shape(self.wd_ind_trg)[-1],
        dtype=tf.float32)
    self.encoder = Encoder(
        num_wds, wd_ind_src, self.input_mask, n_layers=n_layers, ndims=ndims)
    self.decoder = Decoder(
        num_wds,
        wd_ind_trg,
        self.output_mask,
        self.encoder,
        n_layers=n_layers,
        ndims=ndims)
    opt = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
    self.prediction_mask = tf.concat((tf.zeros(
        (tf.shape(self.output_mask)[0], 1)), self.output_mask[:, :-1] - self.output_mask[:, 1:]),
                                     1)
    self.loss = tf.reduce_mean(
        tf.square(
            tf.reduce_max(
                tf.nn.sparse_softmax_cross_entropy_with_logits(
                    labels=self.wd_ind_trg, logits=self.decoder.output_raw) *
                self.prediction_mask, 1)))
    self.optimizer, self.grad_norm_total = nn_utils.apply_clipped_optimizer(
        opt, self.loss)


# In[71]:

transformer = Transformer(num_wds_input)
MAX_LEN = 20

sess = tf.Session()
sess.run(tf.global_variables_initializer())
if real:
  for itr, train_batch in enumerate(tqdm.tqdm_notebook(train_it)):
    src_tensor = train_batch.src[0].data.cpu().numpy().transpose()
    src_len = train_batch.src[1].cpu().numpy()
    trg_tensor = train_batch.trg[0].data.cpu().numpy().transpose()
    trg_len = train_batch.trg[1].cpu().numpy()
    src_tensor, trg_tensor = [t[:, :MAX_LEN] for t in [src_tensor, trg_tensor]]
    src_len, trg_len = [np.clip(t, 0, MAX_LEN) for t in [src_len, trg_len]]
    trg_len = np.ceil(
        np.random.uniform(size=trg_len.shape[0]) * (trg_len - 1)).astype(int)
    trn_feed_dict = {
        transformer.wd_ind_src: src_tensor,
        transformer.input_lengths: src_len,
        transformer.wd_ind_trg: trg_tensor,
        transformer.output_lengths: trg_len,
        transformer.learning_rate: 1e-2 / (np.sqrt(itr + 3))
    }
    _, loss = sess.run([transformer.optimizer, transformer.loss],
                       trn_feed_dict)
    if itr % 1000 == 0:
      print(itr, loss)
    if itr > 20000:
      break
else:

  src_tensor = np.random.randint(low=0, high=num_wds_input, size=(BATCH_SIZE, 81))
  src_len = np.random.randint(2, 81, BATCH_SIZE)
  trg_tensor = np.random.randint(low=0, high=num_wds_input, size=(BATCH_SIZE, 84))
  trg_len = np.random.randint(2, 84, BATCH_SIZE)

  fd = {
      transformer.wd_ind_src: src_tensor,
      transformer.wd_ind_trg: trg_tensor,
      transformer.input_lengths: src_len,
      transformer.output_lengths: trg_len,
      transformer.learning_rate: 1e-2}
  sess.run([transformer.optimizer, transformer.loss], fd)
# In[75]:


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


A Jupyter Widget

0 64.42255
1000 5.29291
2000 8.302757
3000 1.4244095
4000 14.254912
5000 7.001434
6000 3.743309
7000 8.526296
8000 2.4261403
9000 2.5368729
10000 7.662856
11000 4.6585245
12000 1.1795835



In [2]:
train_batch.trg[1].cpu().numpy().shape

(4,)

In [3]:
trg_len

array([11,  1,  5,  1])

In [4]:
np.ceil(
        np.random.uniform(size=trg_len.shape[0]) * (trg_len - 1)).astype(int)

array([6, 0, 1, 0])

In [5]:
NUM_MAX_DECODE = MAX_LEN

In [6]:

src_len_decode = src_len[0:1]
src_decode = src_tensor[0:1, :src_len_decode[0]]
autoregressive = trg_tensor[0:1, 0:1]


In [7]:
def detect_end(choice, eos_token=None):
    return choice.flatten()[0] == eos_token

In [8]:
np.ones(autoregressive.shape[1])

array([1.])

In [16]:
for dec_idx in range(10):
    pred = sess.run(
        transformer.decoder.outsoftmax.output[:,-1,:], {
            transformer.wd_ind_src: src_decode,
            transformer.input_lengths: src_len_decode,
            transformer.wd_ind_trg: autoregressive,
            transformer.output_lengths: np.ones(1)*autoregressive.shape[1],
        })
    choice = pred.argmax(1)
    autoregressive = np.concatenate((autoregressive, np.expand_dims(choice, 0)), -1)
    if detect_end(choice, None):
        break

In [17]:
pred.argmax(1)

array([0])

In [18]:
pred

array([[0.00593889, 0.00026155, 0.00024181, ..., 0.00015679, 0.00019054,
        0.00029458]], dtype=float32)

In [19]:
translation = [EN.vocab.itos[a] for a in autoregressive.flatten()]

In [20]:
translation

['<bos>',
 'it',
 'actually',
 'it',
 '<unk>',
 'it',
 'it',
 'with',
 '<unk>',
 '<unk>',
 '<unk>',
 '<unk>']

In [21]:
pred[:,12:20]

array([[0.00042623, 0.00035116, 0.00084691, 0.0001743 , 0.00045299,
        0.00058221, 0.00027083, 0.00034129]], dtype=float32)